In [1]:
# Importing tval is from the parent directory of notebooks. The parent is appended
# to PYTHONPATH
import os, sys
sys.path.append("..")

In [19]:
import shapely
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon
from shapely.ops import split
import geopandas as gpd
import pandas as pd
from tval import trace_builder
from tval import trace_validator
from typing import Union

In [31]:
ls1 = LineString([(0,0), (0, 5)])
ls2 = LineString([(-2, 4), (0.001, 2.5), (-2, 0)])
ls3 = LineString([(0.000001, 1), (-2, 2), (0.0000001, 3)])
ms = MultiLineString([
    LineString([(0.00001,0), (0.000001, 1), (2, 2)]),
    LineString([(2,6), (3, 1)]),
])
ls1_buffer = ls1.buffer(0.001)
ls1.overlaps(ms)

False

In [21]:
def segmentize_ls(ls):
    points = [Point(c) for c in ls.coords]
    segmentized = []
    for idx, p in enumerate(points):
        if idx == len(points) -1:
            break
        else:
            segmentized.append(LineString([p, points[idx+1]]))
    return segmentized

In [22]:
def segment_within_buffer(ls_buffer, ms):
    all_segments = []
    for ls in ms:
        all_segments.extend(segmentize_ls(ls))
    for seg in all_segments:
        if seg.within(ls_buffer):
            return True
    return False

In [23]:
segment_within_buffer(ls1_buffer, ms)

True

In [27]:
def split_to_determine_triangle_errors(trace, splitter_trace, snap_threshold, snap_threshold_triangle_multiplier):
    assert isinstance(trace, LineString)
    assert isinstance(splitter_trace, LineString)
    segments = split(trace, splitter_trace)
    if len(segments) > 2:
        seg_lengths = [seg.length for seg in segments]
        for seg_length in seg_lengths:
            if snap_threshold < seg_length < snap_threshold * snap_threshold_triangle_multiplier:
                return True
    return False
    

In [32]:
split_to_determine_triangle_errors(ls1, ls3, 0.001, 10)

False